In [4]:
x

array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.1543238e+00,
       6.7145781e+03, 1.1617169e+00], dtype=float32)

In [3]:
import IPython.display as ipd
import numpy as np
with open('audio.bin', 'rb') as fp:
    x = np.frombuffer(fp.read(), dtype=np.float32)
ipd.Audio(data=x, rate=44100)

/home/codespace/.local/lib/python3.10/site-packages/IPython/lib/display.py:172: RuntimeWarning: invalid value encountered in cast
  data = np.array(data, dtype=float)
/home/codespace/.local/lib/python3.10/site-packages/IPython/lib/display.py:188: RuntimeWarning: invalid value encountered in cast
  return scaled.astype("<h").tobytes(), nchan
